In [1]:
# apattion to: https://www.strollswithmydog.com/raw-file-conversion-steps/


In [2]:
%matplotlib inline
import matplotlib as mlp
mlp.rcParams['figure.figsize'] = 10, 15
import matplotlib.pyplot as plt
import rawpy
import numpy as np

In [3]:
# read dng file
raw = rawpy.imread("./hq_cam_color_checker_new.dng")
# get the bayer matrix
bayer = raw.raw_image_visible.copy()
# close the raw object
raw.close()

In [4]:
# step 1: black level correction
# step 2: White Balance the data
# step 3: Correct linear brightness
# step 4: Properly Clip image data
# step 5: Demosaic it
# step 6: Apply Color Transforms and Corrections
# step 7: Apply Gamma

In [5]:
# step 1: black level correction
# start linear conversion

# black level correction
black_level = 256

# substract black level from bayer matrix
bayer = bayer-256

# normalize bayer data
#bayer = bayer/(2**12)

In [6]:
print(f"max:{np.max(bayer)}, min:{np.min(bayer)}")

max:2695, min:2


In [7]:
# for step 2  
def get_bayer_color(bayer, y1, y2, x1, x2):
    red = []
    green = []
    blue = []
    for y in range(y1, y2):
        for x in range(x1, x2):
            if y%2==0:
                if x%2==0:
                    blue.append(bayer[y][x])
                else:
                    green.append(bayer[y][x])
            else:
                if x%2==0:
                    green.append(bayer[y][x])
                else:
                    red.append(bayer[y][x])
    
    return red, green, blue

In [8]:
# for step 2  
def correct_bayer_wb(bayer, wb):
    # wb: [red, green, blue, green]
    for y in range(bayer.shape[0]):
        for x in range(bayer.shape[1]):
            if y%2==0:
                if x%2==0:
                    # blue
                    bayer[y][x] = bayer[y][x]*wb[2]
                else:
                    # green
                    bayer[y][x] = bayer[y][x]*wb[1]
            else:
                if x%2==0:
                    # green
                    bayer[y][x] = bayer[y][x]*wb[1]
                else:
                    # red
                    bayer[y][x] = bayer[y][x]*wb[0]
    
    return bayer
    

In [9]:
# step 2: White Balance the data

# white balance
# correct the white blance based on a gray card
# in the raw image

# define position of gray card
#[2300,2800, 2050,2550]
y1 = 2300
y2 = 2800
x1 = 2050
x2 = 2550

red, green, blue = get_bayer_color(bayer, y1, y2, x1, x2)

red_median = np.median(red)
green_median = np.median(green)
blue_median = np.median(blue)
print(red_median, green_median, blue_median)

# wb: [red, green, blue, green]
base_wb = [green_median/red_median, 1.0, green_median/blue_median, 1.0]
print(base_wb)

# multiply every pixel with the corresponding base wb value
bayer_corr = correct_bayer_wb(bayer, base_wb)
bayer_corr

129.0 524.0 383.0
[4.062015503875969, 1.0, 1.3681462140992167, 1.0]


array([[1663, 1688, 1663, ..., 1400, 1254, 1464],
       [1642, 1563, 1653, ..., 1470, 1375, 1372],
       [1618, 1713, 1637, ..., 1394, 1384, 1374],
       ...,
       [1399, 1356, 1347, ..., 1141, 1098, 1035],
       [1325, 1442, 1284, ..., 1145, 1039, 1058],
       [1386, 1344, 1305, ..., 1080, 1053, 1023]], dtype=uint16)

In [10]:
# step 3: Correct linear brightness

# correct linear brightness
# output_bps=16
# current dtype uint 16

# migth not be necessary

In [11]:
# step 4: Properly Clip image data

# no clipping needed

In [12]:
# for step 5
def demosaic_raw_rgb(bayer):
    rgb = np.zeros([bayer.shape[0],bayer.shape[1],3]) 
    for y in range(bayer.shape[0]):
        for x in range(bayer.shape[1]):
            # marginal condition
            if y==3039 or x==4055:
                if y%2==0:
                    if x%2==0:
                        # blue
                        g = bayer[y-1][x]+bayer[y][x-1]
                        rgb[y][x] = [bayer[y-1][x-1],g/2,bayer[y][x]]                  
                    else:
                        # green
                        g = bayer[y][x]+bayer[y-1][x-1]
                        rgb[y][x] = [bayer[y-1][x],g/2,bayer[y][x-1]]                    
                else:
                    if x%2==0:
                        # green
                        g = bayer[y][x]+bayer[y-1][x-1]
                        rgb[y][x] = [bayer[y][x-1],g/2,bayer[y-1][x]] 
                    else:
                        g = bayer[y-1][x]+bayer[y][x-1]
                        rgb[y][x] = [bayer[y][x],g/2,bayer[y-1][x-1]]  
                        # red
            else:
                if y%2==0:
                    if x%2==0:
                        # blue
                        g = bayer[y+1][x]+bayer[y][x+1]
                        rgb[y][x] = [bayer[y+1][x+1],g/2,bayer[y][x]]                  
                    else:
                        # green
                        g = bayer[y][x]+bayer[y+1][x+1]
                        rgb[y][x] = [bayer[y+1][x],g/2,bayer[y][x+1]]                    
                else:
                    if x%2==0:
                        # green
                        g = bayer[y][x]+bayer[y+1][x+1]
                        rgb[y][x] = [bayer[y][x+1],g/2,bayer[y+1][x]] 
                    else:
                        g = bayer[y+1][x]+bayer[y][x+1]
                        rgb[y][x] = [bayer[y][x],g/2,bayer[y+1][x+1]]  
                        # red
                    
    return rgb

In [13]:
# step 5: Demosaic it

# take four bayerpixel to one RGB pixel
# R, G stay constant G is average of both green channels

rgb_array = demosaic_raw_rgb(bayer)
rgb_array

array([[[1563. , 1665. , 1663. ],
        [1563. , 1670.5, 1663. ],
        [1608. , 1645. , 1663. ],
        ...,
        [1470. , 1387.5, 1254. ],
        [1372. , 1419.5, 1254. ],
        [1023. , 1258.5, 1254. ]],

       [[1563. , 1677.5, 1618. ],
        [1563. , 1683. , 1637. ],
        [1608. , 1675. , 1637. ],
        ...,
        [1470. , 1384.5, 1384. ],
        [1372. , 1374.5, 1384. ],
        [1372. , 1419.5, 1254. ]],

       [[1718. , 1690.5, 1618. ],
        [1718. , 1725. , 1637. ],
        [1677. , 1717. , 1637. ],
        ...,
        [1279. , 1409. , 1384. ],
        [1344. , 1399. , 1384. ],
        [1372. , 1374.5, 1384. ]],

       ...,

       [[1356. , 1420.5, 1325. ],
        [1356. , 1394.5, 1284. ],
        [1413. , 1365.5, 1284. ],
        ...,
        [1141. , 1121.5, 1039. ],
        [1035. , 1078. , 1039. ],
        [1035. , 1069. , 1050. ]],

       [[1344. , 1414. , 1325. ],
        [1344. , 1373.5, 1284. ],
        [1502. , 1344.5, 1284. ],
        .

In [14]:
# step 6: Apply Color Transforms and Corrections

# apply color transformation
# output_color=ColorSpace.sRGB
# maybe create other color spaces as well like HSV and XYZ

# forward matrix
# raw white-balanced demosaiced data: rgb to XYZ
# table from dng meta data
CM = np.array([[0.3806826174, -0.09813511372, -0.008731900714 ],
                [-0.220981434, 0.888884902, 0.294952035 ],
                [0.06568053365, 0.1024207622, 0.5077140927]])

XYZ = np.zeros(rgb_array.shape)
for y in range(rgb_array.shape[0]):
    for x in range(rgb_array.shape[1]):
        pix = rgb_array[y,x,:]
        XYZ_pix = CM.dot(pix)
        XYZ[y,x,:] = XYZ_pix
        
XYZ

array([[[ 417.09081577, 1625.10461469, 1117.51777932],
        [ 416.55107264, 1629.99348165, 1118.08109351],
        [ 436.18423582, 1597.38275212, 1118.42498809],
        ...,
        [ 412.4911738 , 1278.35494543,  875.33266426],
        [ 372.04395365, 1328.45544283,  872.17343636],
        [ 254.98547349, 1262.46749407,  832.7611874 ]],

       [[ 416.25706238, 1622.94283439, 1095.95090467],
        [ 415.55141314, 1633.43579002, 1106.16078663],
        [ 433.46721183, 1616.38054627, 1108.29704454],
        ...,
        [ 411.65043204, 1314.03205528,  941.02823403],
        [ 375.32488668, 1326.79938679,  933.56733411],
        [ 372.04395365, 1328.45544283,  872.17343636]],

       [[ 473.98711159, 1600.24621585, 1107.4628573 ],
        [ 470.43554406, 1636.51683363, 1120.64294136],
        [ 455.61263765, 1638.46599321, 1117.13067338],
        ...,
        [ 336.53574183, 1378.01718927,  930.99256077],
        [ 362.2614631 , 1354.76454704,  934.23758784],
        [ 375.32488668

In [15]:
# step 6.2

# convert XYZ to sRGB
TM = np.array([[3.2406, -1.5372, -0.4986],
                [-0.9689, 1.8758, 0.0415],
                [0.0557, -0.2040, 1.0570]])

sRGB = np.zeros(XYZ.shape)
for y in range(XYZ.shape[0]):
    for x in range(XYZ.shape[1]):
        pix = XYZ[y,x,:]
        sRGB_pix = TM.dot(pix)
        sRGB[y,x,:] = sRGB_pix
        
abs(sRGB)

array([[[1703.68068091, 2690.62893269,  872.92690978],
        [1713.22580723, 2700.34580399,  872.49494033],
        [1599.64483102, 2620.16629735,  880.60459291],
        ...,
        [1064.80919072, 2034.60181392,  687.41797564],
        [1271.32174588, 2167.63853058,  671.60526011],
        [1529.57383454, 2155.6406894 ,  636.88789716]],

       [[1692.30620976, 2686.48666356,  850.5252864 ],
        [1715.81335522, 2707.27676337,  859.13726401],
        [1632.60323549, 2658.01457451,  865.87246834],
        ...,
        [1155.13236278, 2105.0658974 ,  749.53323316],
        [1288.7548624 , 2163.90105141,  737.01919344],
        [1271.32174588, 2167.63853058,  671.60526011]],

       [[1476.07682982, 2588.45544784,  870.53909425],
        [1549.91282315, 2660.47995996,  876.87341476],
        [1599.19296493, 2678.35234839,  871.93768306],
        ...,
        [1491.90318916, 2297.45135464,  721.68867095],
        [1374.41042568, 2229.04306564,  731.29512625],
        [1288.7548624 

In [16]:
# step 7: Apply Gamma

# start non linear conversion
# apply gamma



In [17]:
np.max(sRGB)

4246.19169980875

In [18]:
np.min(sRGB)

-3100.1004530237483

In [19]:
np.median(sRGB)

229.26505948775218

In [21]:
np.median(abs(sRGB))

365.427807914632